# Reproducing KeyClass: An Automatic Weak Supervision Model for Classifying Unstructured Clinical Notes

## Team ID: 62, Paper: 18
* Minjia Zhu (minjiaz2@illinois.edu)
* Eddie Rodriguez (er17@illinois.edu)
* Yong Yang (yongy3@illinois.edu)
---
#### **Github Repo:** https://github.com/MinjiaZhu/Tagging-Clinical-Notes-Using-KeyClass.git
#### **Video Presenation:** https://youtu.be/rDslaPhIbck


# 1. Introduction

The Electronic Health Record (EHR) system has revolutionized the way patient data is stored and accessed within the healthcare sector. With the increasing adoption of EHR systems, there is a growing need for efficient and accurate methods to process and analyze the vast amount of unstructured clinical text contained within these records. One particular challenge is the assignment of International Classification of Diseases (ICD) codes to patient diagnoses, a process that is currently manual, time-consuming, and prone to errors. ICD-9 -- the ninth revision of the ICD codes -- defines more than 14,000 unique codes, and often times patients are assigned more than one code when being diagnosed (WHO, 1988). The issue boils down to a multiclass and multilabel machine learning task.

As mentioned in the article "Artificial Intelligence in Helathcare", the use of AI applications may reduce the United States' annual healthcare costs by approximately 150 billion USD come 2026. By streamlining something as ubiquitous as clinical text into patient diagnoses without the need for human intervention, the healthcare sector could better utilize their workforce on tasks that are less menial.

The paper "Classifying Unstructured Clinical Notes via Automatic Weak Supervision" (Gao et al., 2022) introduces KeyClass, a novel weakly supervised text classification framework that leverages pre-trained language models and data programming to automatically assign high-level ICD-9 codes to clinical notes. This approach eliminates the need for human-labeled data via training signals that include descriptors of the ICD codes, making the classification process more efficient and scalable. Previous attempts of weak supervision relied upon manually created rule-based labeling functions. Whereas KeyClass uses keyword-matching from referenced datasets such as IMDb movie reviews with both negative and positive target keywords to train the model's labeling function with little to no human intervention.

KeyClass was shown to outperform the weakly supervised model, FasTag (Venkataraman et el., 2020), in assigning ICD-9 codes,  measured by metrics such as recall by 12.8% (0.896±0.0009), precision by 16.3% (0.507±0.0016), and F1-score by 19.1% (0.6252±0.0014). The KeyClass paper also considered the fully supervised FasTag model, where KeyClass demonstrated similar performance in terms of recall, precision, and F1 score. With some further exploration, KeyClass may be the definitive solution to the clinical note transcription problem. In the following sections we plan to implement, analyze, and interpret the results of the KeyClass model.

# 2. Scope of Reproducibility

The basis for this paper was to attempt to reproduce the work of Gao et al. (2022) by classifying clinical notes (EVENTNOTES and DIAGNOSES_ICD) obtained from the MIMIC-III database into viable ICD-9 codes. In order to harness KeyClass into a viable model for classification of clinical notes, there is a dependency on it's precedessor FasTag (Venkataraman et al. 2020), a fully supervised automatic text classification model. FasTag trains long short-term memory (LSTM) recurrent neural networks (RNNs) on unstructured clinical notes.

For the purpose of our KeyClass model reproduction, we relied on FasTag to prepare test label and training label text files from the raw MIMIC-III data. Preprocessing via FasTag took raw clinical notes that originated in the form of large comma separated variable (CSV) files and created useable training and validation data sets that would be comparable to the format of the datasets readily available from the KeyClass GitHub repository.

Once we obtained all of our datasets, we passed them through KeyClass's labeling functions to generate labels. These labels would be the crux for training the downstream classifier. The following were the hypotheses we set out to prove:


#### Hypothesis 1:
KeyClass will significantly reduce the time and resources required for coding clinical notes while maintaining or improving accuracy compared to manual coding.

#### Hypothesis 2:
Verify the efficacy of the framework by sampling datasets via the MIMIC-III database and compare the results to human-labeled documents. Compare time and resources required for coding the clinical notes and see how it compares to the accuracy of KeyClass with manual coding and labeling.

#### Hypothesis 3:
The framework can generalize across different sets of clinical notes and coding systems with minimal adjustments.


# 3. Methodology


##3.1 Compute Environment
We begin by establishing our connection to our virtual machine, connecting our VM to the model and data from our team's GitHub. We must then download the necessary model dependencies used throughout the entire classification model. Our environment uses Python version 3.10.

In [ ]:
!pip install torch torchvision torchaudio
!pip install sentence_transformers

In [ ]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())


CUDA Available: True
Number of GPUs: 1


In [ ]:
!git clone https://github.com/MinjiaZhu/Tagging-Clinical-Notes-Using-KeyClass.git

Cloning into 'Tagging-Clinical-Notes-Using-KeyClass'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 81 (delta 16), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (81/81), 269.12 KiB | 9.61 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [ ]:
import os
import sys
repo_root = "/content/Tagging-Clinical-Notes-Using-KeyClass"

for root, dirs, files in os.walk("/content/Tagging-Clinical-Notes-Using-KeyClass"):
    sys.path.append(root)  # Adds all directories and subdirectories

# Check current sys.path entries
print(sys.path)

['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython', '/content/Tagging-Clinical-Notes-Using-KeyClass', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/logs', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/logs/refs', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/logs/refs/heads', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/logs/refs/remotes', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/logs/refs/remotes/origin', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/refs', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/refs/tags', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/refs/heads', '/content/Tagging-Clinical-Notes-Using-KeyClass/.git/refs/remotes', '/content/Tagg

We took createAdmissionNoteTable.R from asTag.

In [ ]:
project_code_dir = os.path.join(repo_root, "project_code")
os.chdir(project_code_dir)
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 16.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [ ]:
import argparse
import label_data, encode_datasets, train_downstream_model
import torch
import pickle
import numpy as np
import os
from os.path import join, exists
from datetime import datetime
from keyclass import models, helper
import create_lfs
import train_classifier
from transformers import AutoTokenizer, AutoModel

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##3.2 Data Description

Data procurement for the reproduction of KeyClass included gaining access to the MIMIC-III dataset via PhysioNet. The MIMIC-III data included various CSV files relating to patient data that was obtained from the Beth Israel Deaconess Medical Center in Boston, MA in the United States. From the many files available, NOTEEVENTS and DIAGNOSES_ICD, are the two files required to be used in the KeyClass classifier (via some preprocessing using FasTag).

NOTEEVENTS is a file containing the raw inputs taken from a patients visit such as medications that are being taken by the patient, health markers such as blood pressure readings, etc. DIAGNOSES_ICD contains the information that diagnoses the patient given their ailments to a specific diagnosis, or multiple diagnoses in terms of ICD codes. We run these two files through FasTag (Venkataraman et al., 2020) to get MIMIC-III data that is of a form compatible with KeyClass.

Next we run the YAML configuration files to obtain the training and validation datasets. These YAML files are called in the *get_data*  shell file which automates our data acquisition from the various data resources that include IMDb movie reviews and Amazon product reviews to name a few, into the folder named *data*. This *data* folder is where all our training and validation text files that are used to ultimately train our downstream classifier will emanate from.


##  3.3 Model Description
KeyClass (Gao et al., 2022) takes clinical notes, finds keywords and phrases in the text tying them to ICD-9 codes that are run through labeling functions. These labeling functions allow the model to produce probabilistic labels that in turn get used to train our downstream classifier.

The process of training begins by running the MIMIC-III training and validation sets, born from the preprocessing stage using FasTag (Venkataraman et el., 2020), through a pretrained model using BERT architecture designed for medical text processing (aka BlueBERT). We encode the data using one to three n-grams,  -- sequential words or tokens -- that get batched into 512 samples used for training, adjusting the model's weights with the standard learning rate of 0.001, and allowing for three epochs of patience (aka early stopping).

As the data is fed foward, we iterate through ten fully connected layers with the following neuron count: 768, 1024, 512, 256, 64, 256, 512, 1024, 64, 17. We then step through a Temporal Convolutional Network (TCN) consisting of two hidden layers with 128 and 64 neurons, respectively.

The model activates hidden layers with Leaky ReLU and trains the model using Binary Cross-Entropy Loss (BCE) function, outputting probabilistic values commonly used for classification. The original authors of KeyClass (Gao et al, 2022) provided a tutorial on their GitHub related to classification of movie reviews into two target classes: good or bad. Simililarly, KeyClass outputs only outputs a single diagnosis label from the input dataset.


#### Original Paper Link: https://proceedings.mlr.press/v182/gao22a/gao22a.pdf
#### Original Github Repo: https://github.com/autonlab/KeyClass

## 3.4 Model Training

## 3.4.1 Model Hyperparameters

The base encoder used for the reproduction and ablations was the BlueBERT PubMed encoder which uses pretrained models using preprocessed text from abstracts on PubMed and clinical notes (MIMIC-III). As mentioned in the Model Description section, our model was tuned to use the activation function of Leaky ReLU at the hidden layers to avoid a proliferation of "dead neurons". The model was trained using Binary Cross-Entropy Loss, which is commonly used when we anticipate imbalanced classes. The model was originally trained for 20 epochs, and this remained unchanged for our reproduction model. Early stopping is another hyperparameter that will be discussed in a  subsequent section, as this was a part of our ablations for KeyClass. A few of the other hyperparameters used in this model were referenced in the previous section.

## 3.4.2 Training Computational Requirements

Our reproduction efforts focused on a small sample, 10% of the original dataset. This allowed us to concentrate on data that could be processed in the Google Colab Pro environment in a timely manner. To ensure further flexibility in training, we provided options for utilizing either a central processing unit (CPU) or graphics processing unit (GPU), with the ability to parallelize data processing using compute unified device architecture (CUDA) through PyTorch.

Our training used a T4 GPU with 51 GB of system RAM, 15 GB of VRAM, and 201 GB of disk storage. The runtime with the aforementioned compute specifications was just under 20 minutes.

In [ ]:
data_dir = os.path.join(repo_root, "data")
os.chdir(data_dir)

train_path = 'icd9NotesDataTable_train_sample.csv'
test_path = 'icd9NotesDataTable_valid_sample.csv'
codeIdx = 9
textIdx = 6
helper.load_and_process_data(train_path, test_path, 'mimic/', max_length=1000, random_state=1234, sample_size=0.1)

In [ ]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [ ]:
config_file_path = r'../config_files/config_mimic_bert.yml' # Using BERT model
random_seed = 0

##3.4.3 Data Embedding & Label Model Training

TFetch text data created in the step above, convert it into numerical embeddings, and store these embeddings for future use.

In [ ]:
args = helper.Parser(config_file_path=config_file_path).parse()

if args['use_custom_encoder']:
    model = models.CustomEncoder(pretrained_model_name_or_path=args['base_encoder'],
        device='cuda' if torch.cuda.is_available() else 'cpu')
else:
    model = models.Encoder(model_name=args['base_encoder'],
        device='cuda' if torch.cuda.is_available() else 'cpu')

for split in ['train', 'test']:
    sentences = helper.fetch_data(dataset=args['dataset'], split=split, path=args['data_path'])
    embeddings = model.encode(sentences=sentences, batch_size=args['end_model_batch_size'],
                                show_progress_bar=args['show_progress_bar'],
                                normalize_embeddings=args['normalize_embeddings'])
    with open(join(args['data_path'], args['dataset'], f'{split}_embeddings.pkl'), 'wb') as f:
        pickle.dump(embeddings, f)

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import gc

del model
gc.collect()
torch.cuda.empty_cache()

Training a label model to categorize text data into multiple classes.

The model loads training text data ("train_text") and optional training labels ("y_train") from specified files.

In [ ]:
args = helper.Parser(config_file_path=config_file_path).parse()

train_text = helper.fetch_data(dataset=args['dataset'], path=args['data_path'], split='train')

training_labels_present = False
if exists(join(args['data_path'], args['dataset'], 'train_labels.txt')):
    with open(join(args['data_path'], args['dataset'], 'train_labels.txt'), 'r') as f:
        y_train = f.readlines()
    y_train = np.array([int(i.replace('\n','')) for i in y_train])
    training_labels_present = True
else:
    y_train = None
    training_labels_present = False
    print('No training labels found!')

with open(join(args['data_path'], args['dataset'], f'train_embeddings.pkl'), 'rb') as f:
    X_train = pickle.load(f)

# Print dataset statistics
print(f"Getting labels for the {args['dataset']} data...")
print(f'Size of the data: {len(train_text)}')
if training_labels_present:
    print('Class distribution', np.unique(y_train, return_counts=True))
class_balance = np.unique(y_train, return_counts=True)[1] / np.unique(y_train, return_counts=True)[1].sum()
# Load label names/descriptions
label_names = []
for a in args:
    if 'target' in a: label_names.append(args[a])

# Creating labeling functions
labeler = create_lfs.CreateLabellingFunctions(custom_encoder=args['use_custom_encoder'],
                                              base_encoder=args['base_encoder'],
                                            device=torch.device(args['device']),
                                            label_model=args['label_model'])

current_length = len(class_balance)
if current_length < 17:
    half_last_value = class_balance[-1] / 2.0
    # print(half_last_value)
    class_balance[-1] = half_last_value
    class_balance = np.append(class_balance, half_last_value)

print(class_balance)

proba_preds = labeler.get_labels(
    text_corpus=train_text,
    label_names=label_names,
    max_df=0.3,
    min_df=0.001,
    ngram_range=(1,1),
    topk=args['topk'],
    y_train=y_train,
    label_model_lr=args['label_model_lr'],
    label_model_n_epochs=args['label_model_n_epochs'],
    verbose=True,
    n_classes=args['n_classes'],
    class_balance=class_balance,
    min_topk=False
)

Getting labels for the mimic data...
Size of the data: 389
Class distribution (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16]), array([  1,  25,  12,   7,  10,  12,  30, 152,  41,  41,   9,   3,   4,
         5,  28,   9]))
0.011568123393316195
[0.00257069 0.06426735 0.03084833 0.01799486 0.02570694 0.03084833
 0.07712082 0.3907455  0.10539846 0.10539846 0.02313625 0.00771208
 0.01028278 0.01285347 0.07197943 0.01156812 0.01156812]
Found assigned category counts [ 632  602 1508 2675  499  601   99 2493 1676   41  661  251  468  125
  386  451  653]
labeler.vocabulary:
 13821
labeler.word_indicator_matrix.shape (389, 510)
Len keywords 510
assigned_category: Unique and Counts (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]), array([30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]))
tuberculosis,  found,  unspecified,  bacilli,  tubercle,  bacteriological,  examination,  histological,  specified,  sputum,  confirmed,  

100%|██████████| 100/100 [10:47<00:00,  6.47s/epoch]


In [ ]:
# Separate probabilities and labels from proba_preds tuple
proba_preds_probabilities, labels_or_other_data = proba_preds

# Generate predictions from probabilities
y_train_pred = np.argmax(proba_preds_probabilities, axis=1)

# Save the predictions
if not os.path.exists(args['preds_path']):
    os.makedirs(args['preds_path'])

# Here you can save both the probabilities and labels_or_other_data if needed
with open(os.path.join(args['preds_path'], f"{args['label_model']}_proba_preds.pkl"), 'wb') as f:
    pickle.dump(proba_preds_probabilities, f)  # Saving the entire tuple to preserve all data

# Print statistics
print('Label Model Predictions: Unique value and counts', np.unique(y_train_pred, return_counts=True))
if training_labels_present:
    print('Label Model Training Accuracy', np.mean(y_train_pred == y_train))

    # Wrap labels in a list if necessary
    y_preds_list = [[label] for label in y_train_pred]
    y_true_list = [[label] for label in y_train]

    training_metrics_with_gt = helper.compute_metrics(y_preds=y_preds_list, y_true=y_true_list, average=args['average'])
    helper.log(metrics=training_metrics_with_gt, filename='label_model_with_ground_truth', results_dir=args['results_path'], split='train')

# OPTIONAL EVALUATION METRICS TO BE ADDED HERE #

Label Model Predictions: Unique value and counts (array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16]), array([15, 35, 15, 15, 17, 10, 19, 62, 16, 31, 13, 11, 13, 26, 79,  5,  7]))
Label Model Training Accuracy 0.10025706940874037
Saving results in ../results/mimic/train_label_model_with_ground_truth_03-May-2024-02_46_54.txt...


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.



In [ ]:
import gc

#del labeler
gc.collect()
torch.cuda.empty_cache()

##3.4.4 Train Model

In [ ]:
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm, trange

def custom_train(model, device, X_train, y_train, epochs, batch_size, lr):
    if isinstance(y_train, np.ndarray):
        y_train = torch.from_numpy(y_train)

    if isinstance(X_train, np.ndarray):
        X_train = torch.from_numpy(X_train)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=lr)
    N = len(X_train)
    pbar = trange(epochs, unit="batch")
    for nep in pbar:
        pbar.set_description(f"Epoch {nep}")
        permutation = torch.randperm(N)
        running_loss = 0.0
        for i in range(0, N, batch_size):
            optimizer.zero_grad()
            indices = permutation[i:i + batch_size]

            batch_x, batch_y = X_train[indices], y_train[indices]
            batch_y = batch_y.to(device)
            batch_x = batch_x.to(device)

            out = model.forward(batch_x, mode='inference', raw_text=False)
            loss = criterion(out, batch_y)
            loss.backward()
            optimizer.step()
            running_loss = running_loss + (loss.cpu().detach().numpy() *
                                           batch_size / N)
        print(running_loss)
    return model


In [ ]:
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
args = helper.Parser(config_file_path=config_file_path).parse()

# Set random seeds
random_seed = random_seed
torch.manual_seed(random_seed)
np.random.seed(random_seed)

X_train_embed_masked, y_train_lm_masked, y_train_masked, \
	X_test_embed, y_test, training_labels_present, \
	sample_weights_masked, proba_preds_masked = train_downstream_model.load_data(args, class_balance=class_balance, max_num=2000)

# X_train_embed_masked = normalize(X_train_embed_masked)
# X_test_embed = normalize(X_test_embed)
# with open(join(args['data_path'], args['dataset'], f'train_embeddings.pkl'), 'rb') as f:
#     X_train = pickle.load(f)
# Train a downstream classifier

if args['use_custom_encoder']:
	encoder = models.CustomEncoder(pretrained_model_name_or_path=args['base_encoder'], device=args['device'])
else:
	encoder = models.Encoder(model_name=args['base_encoder'], device=args['device'])
classifier = models.FeedForwardFlexible(encoder_model=encoder,
										h_sizes=[768, 1024, 512, 256, 64, 256, 512, 1024, 64, 17],
										activation=eval(args['activation']),
										device=torch.device(args['device']))

# classifier = models.FeedForwardTCN(encoder_model=encoder,
# 										input_size=768,
# 										output_size=768,
# 										kernel_size=3,
# 										stride=1,
# 										dilation=1,
# 										padding=2,
# 										dropout=0.1,
# 										device=torch.device(args['device']))
print('\n===== Training the downstream classifier =====\n')
X_train_embed_masked = X_train_embed_masked.astype(np.float32)
y_train_lm_masked = y_train_lm_masked.astype(np.float32)

X_train_embed_masked, y_train_lm_masked = shuffle(X_train_embed_masked, y_train_lm_masked, random_state=2)
model = train_classifier.train(model=classifier,
							device=torch.device(args['device']),
							X_train=X_train_embed_masked,
							y_train=y_train_lm_masked,
							sample_weights=sample_weights_masked if args['use_noise_aware_loss'] else None,
							epochs=args['end_model_epochs'],
							batch_size=512,
							criterion=eval(args['criterion']),
							raw_text=False,
							lr=1,
							weight_decay=0,
							patience=args['end_model_patience'])

end_model_preds_train = model.predict_proba(torch.from_numpy(X_train_embed_masked), batch_size=512, raw_text=False)
end_model_preds_test = model.predict_proba(torch.from_numpy(X_test_embed), batch_size=512, raw_text=False)

Confidence of least confident data point of class 0: 0.9742685415447618
Confidence of least confident data point of class 1: 0.01697309023489411
Confidence of least confident data point of class 2: 0.029033721457734765
Confidence of least confident data point of class 3: 0.022977055231414013
Confidence of least confident data point of class 4: 0.02419476788144564
Confidence of least confident data point of class 5: 0.029033721457734765
Confidence of least confident data point of class 6: 0.010920938886653873
Confidence of least confident data point of class 7: 1.2289455332621385e-13
Confidence of least confident data point of class 8: 0.0035039073015979945
Confidence of least confident data point of class 9: 0.00592023895541882
Confidence of least confident data point of class 10: 0.021775291093301077
Confidence of least confident data point of class 11: 0.2861958184241912
Confidence of least confident data point of class 12: 0.10915273413128912
Confidence of least confident data point

Epoch 1:   5%|▌         | 1/20 [00:00<00:03,  5.36batch/s, best_loss=inf, running_loss=0.214, tolerance_count=0]

0.21436229401811535


Epoch 4:  20%|██        | 4/20 [00:00<00:00, 16.58batch/s, best_loss=0.214, running_loss=10.2, tolerance_count=3]

10.524280381386568
10.524279753775707
10.22854198465617
10.22854198465617
Stopping early...


In [ ]:
import gc

del labeler
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
args = helper.Parser(config_file_path=config_file_path).parse()

# Set random seeds
random_seed = random_seed
torch.manual_seed(random_seed)
np.random.seed(random_seed)

X_train_embed_masked, y_train_lm_masked, y_train_masked, \
	X_test_embed, y_test, training_labels_present, \
	sample_weights_masked, proba_preds_masked = train_downstream_model.load_data(args, class_balance=class_balance, max_num=2000)

# X_train_embed_masked = normalize(X_train_embed_masked)
# X_test_embed = normalize(X_test_embed)
# with open(join(args['data_path'], args['dataset'], f'train_embeddings.pkl'), 'rb') as f:
#     X_train = pickle.load(f)
# Train a downstream classifier

if args['use_custom_encoder']:
	encoder = models.CustomEncoder(pretrained_model_name_or_path=args['base_encoder'], device=args['device'])
else:
	encoder = models.Encoder(model_name=args['base_encoder'], device=args['device'])
classifier = models.FeedForwardFlexible(encoder_model=encoder,
										h_sizes=[768, 1024, 512, 256, 64, 256, 512, 1024, 64, 17],
										activation=eval(args['activation']),
										device=torch.device(args['device']))

# classifier = models.FeedForwardTCN(encoder_model=encoder,
# 										input_size=768,
# 										output_size=768,
# 										kernel_size=3,
# 										stride=1,
# 										dilation=1,
# 										padding=2,
# 										dropout=0.1,
# 										device=torch.device(args['device']))

print('\n===== Training the downstream classifier =====\n')
X_train_embed_masked = X_train_embed_masked.astype(np.float32)
y_train_lm_masked = y_train_lm_masked.astype(np.float32)
X_train_embed_masked, y_train_lm_masked = shuffle(X_train_embed_masked, y_train_lm_masked, random_state=2)
model = train_classifier.train(model=classifier,
							device=torch.device(args['device']),
							X_train=X_train_embed_masked,
							y_train=y_train_lm_masked,
							sample_weights=sample_weights_masked if args['use_noise_aware_loss'] else None,
							epochs=args['end_model_epochs'],
							batch_size=512,
							criterion=eval(args['criterion']),
							raw_text=False,
							lr=1,
							weight_decay=0,
							patience=args['end_model_patience'])

end_model_preds_train = model.predict_proba(torch.from_numpy(X_train_embed_masked), batch_size=512, raw_text=False)
end_model_preds_test = model.predict_proba(torch.from_numpy(X_test_embed), batch_size=512, raw_text=False)

Confidence of least confident data point of class 0: 0.9742685415447618
Confidence of least confident data point of class 1: 0.01697309023489411
Confidence of least confident data point of class 2: 0.029033721457734765
Confidence of least confident data point of class 3: 0.022977055231414013
Confidence of least confident data point of class 4: 0.02419476788144564
Confidence of least confident data point of class 5: 0.029033721457734765
Confidence of least confident data point of class 6: 0.010920938886653873
Confidence of least confident data point of class 7: 1.2289455332621385e-13
Confidence of least confident data point of class 8: 0.0035039073015979945
Confidence of least confident data point of class 9: 0.00592023895541882
Confidence of least confident data point of class 10: 0.021775291093301077
Confidence of least confident data point of class 11: 0.2861958184241912
Confidence of least confident data point of class 12: 0.10915273413128912
Confidence of least confident data point

Epoch 4:  20%|██        | 4/20 [00:00<00:00, 48.90batch/s, best_loss=0.214, running_loss=10.2, tolerance_count=3]

0.21436229401811535
10.524280381386568
10.524279753775707
10.22854198465617
10.22854198465617
Stopping early...


In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=1, n_iter=15000, metric="cosine")
embs =tsne.fit_transform(X_train_embed_masked)
embs

array([[ 1.93343723e+00,  2.51058292e+00],
       [-3.04439211e+00, -1.36360388e+01],
       [-1.40262680e+01, -2.71736169e+00],
       [-3.79357886e+00, -1.18329248e+01],
       [ 5.09500542e+01, -2.19787884e+00],
       [-8.54631543e-01,  4.32944059e+00],
       [ 8.56905162e-01,  8.55604076e+00],
       [-5.58707380e+00, -8.03093052e+00],
       [-9.98862648e+00, -1.21484003e+01],
       [-9.86755610e-01,  1.05322618e+01],
       [-1.14480858e+01, -5.46809971e-01],
       [-1.59591305e+00, -1.54130611e+01],
       [ 5.12629204e+01, -2.22168875e+00],
       [-5.69994307e+00, -1.36705856e+01],
       [-3.57414198e+00,  8.26593018e+00],
       [ 1.30881250e+00,  1.23897686e+01],
       [-3.96336174e+00, -1.02666588e+01],
       [-7.60941505e+00,  7.47535181e+00],
       [ 7.32823086e+00,  6.33896255e+00],
       [ 4.65276146e+00,  5.39088535e+00],
       [-4.58996010e+00,  4.48465252e+00],
       [-2.21892405e+00,  6.65968084e+00],
       [-5.00037336e+00, -1.00915709e+01],
       [-2.

In [ ]:
import numpy as np
import plotly.express as px

# Print the shape to understand the dimensionality
print("Shape of embeddings:", embs.shape)
print("Shape of y_train_lm_masked:", y_train_lm_masked.shape)

# Assuming embs is correctly sized and y_train_lm_masked should be 1D:
if y_train_lm_masked.ndim != 1:
    # Correct or adjust y_train_lm_masked, assuming you need the first dimension
    y_train_lm_masked = y_train_lm_masked[:, 0]  # Adjust this based on your specific case

# Convert labels to string for coloring
labels_str = y_train_lm_masked.astype(str)

# Create the scatter plot
fig = px.scatter(x=embs[:,0], y=embs[:,1], color=labels_str)
fig.show()


Shape of embeddings: (389, 2)
Shape of y_train_lm_masked: (389, 17)


In [ ]:
with open(
        join(args['data_path'], args['dataset'], f'train_embeddings.pkl'),
        'rb') as f:
    X_train_embed = pickle.load(f)
with open(join(args['data_path'], args['dataset'], f'test_embeddings.pkl'),
          'rb') as f:
    X_test_embed = pickle.load(f)

model = train_classifier.self_train(model=model,
									X_train=X_train_embed,
									X_val=X_test_embed,
									y_val=y_test,
									device=torch.device(args['device']),
									lr=eval(args['self_train_lr']),
									weight_decay=eval(args['self_train_weight_decay']),
									patience=args['self_train_patience'],
									batch_size=args['self_train_batch_size'],
									q_update_interval=args['q_update_interval'],
									self_train_thresh=eval(args['self_train_thresh']),
									print_eval=True,
                  raw_text=False)

X_test_embed = torch.from_numpy(X_test_embed)
end_model_preds_test = model.predict_proba(X_test_embed, batch_size=args['self_train_batch_size'], raw_text=False)


0batch [00:00, ?batch/s]


In [ ]:
torch.save(model.state_dict(), 'self_train_model.pt')

#3.5 Model Evaluation

## 3.5.1 Metric Descriptions
We evaluated our model against the original model in the following three evaluation metrics: recall, precision, and F1. These three metrics are what the original paper used to evaluate their model against the weakly supervised model FasTag and the fully supervised model of FasTag.

A review of those three metrics are as follow:

Recall is the ratio of true positives to true positives plus false negatives. In other words, we want to ensure that our model is able to correctly identify all of the positive cases present for a diagnosis, and this measures how many of those positive cases the model was able to correctly identify.

Precision is the ratio of true positives to true positives plus false postives. This metric checks if the positive cases that were predicted by our model actually should have been positive.

F1-Score, or simply F1, is the harmonic mean of the recall and precision metrics. This ensures that we don't overvalue either recall or precision, and a high value of F1 means there is a nice balance between the two metrics.


## 3.5.2 Metric Evaluation Code
The code used to evaluate our model is as follows below and may be incorporated into the section directly following the labeling functions. The section has a comment "# OPTIONAL EVALUATION METRICS TO BE ADDED HERE #", beginning at cell 27 of that code block. The model evaluation metrics will be disucssed in the following section.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Separate probabilities and labels from proba_preds tuple
proba_preds_probabilities, labels_or_other_data = proba_preds

# Obtain predicitions
y_train_pred = np.argmax(proba_preds_probabilities, axis=1)

# Save predictions to the following path
if not os.path.exists(args['preds_path']):
    os.makedirs(args['preds_path'])

with open(os.path.join(args['preds_path'], f"{args['label_model']}_proba_preds.pkl"), 'wb') as f:
    pickle.dump(proba_preds_probabilities, f)  # Saving the entire tuple to preserve all data

print('Label Model Predictions: Unique value and counts', np.unique(y_train_pred, return_counts=True))
if training_labels_present:
    print('Label Model Training Accuracy', np.mean(y_train_pred == y_train))

    y_preds_list = [[label] for label in y_train_pred]
    y_true_list = [[label] for label in y_train]

    training_metrics_with_gt = helper.compute_metrics(y_preds=y_preds_list, y_true=y_true_list, average=args['average'])
    helper.log(metrics=training_metrics_with_gt, filename='label_model_with_ground_truth', results_dir=args['results_path'], split='train')

precision = precision_score(y_true_list, y_preds_list, average=args['average'], zero_division=0)
recall = recall_score(y_true_list, y_preds_list, average=args['average'], zero_division=0)
f1 = f1_score(y_true_list, y_preds_list, average=args['average'], zero_division=0)

print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

additional_metrics = {
    "Precision": precision,
    "Recall": recall,
    "F1-Score": f1,
}

helper.log(
    metrics=additional_metrics,
    filename='additional_metrics',
    results_dir=args['results_path'],
    split='train'
)

# 4. Results

## 4.1 Reproduction of KeyClass

Our attempt to recreate the results of the original KeyClass paper did not prove to be successful. Overall our iteration of the KeyClass model severely underperformed the original paper's results, as well as those of the weakly supervised FasTag (Venkataraman et al., 2020) as shown below in Table 1. Precision for our model was only slightly over 18%, meanwhile recall hovered just above 10%.

![](https://raw.githubusercontent.com/MinjiaZhu/Tagging-Clinical-Notes-Using-KeyClass/main/Accuracy%20Tables%20-%20Table%201.jpg)

We strongly suspect the the vast underperformance of our model was due training on a substantially smaller dataset than was likely used in the original KeyClass. It should also be noted that our model did not have access to the same multilabel set up that the original paper had. These two discrepancies during model training are likely largely at fault for the low precision and recall results.

Taking a step to review our results and compare them to our three original hypotheses posed in the Scope of Reproducibility section of this paper, it is evident that with the appropriate dataset at our disposal we may likely have seen more comparable results with that of the original paper. Given the abridged runtime of this model with access to a GPU, we believe KeyClass may significantly reduce the time required to code clinical notes, we just were unable to prove that directly.

The provided code in the orignal paper's GitHub repository seemed to only support weakly supervised text processing for the multiclass classification problem. This is an oversimplification of the underlying problem which intends to take unstructured, multilabel clinical notes and convert them into viable ICD-9 diagnoses via an automated multiclass classification model. While we weren't able to access any human-labeled documents from a professional healthcare provider, we did spend some time attempting to pair the data from NOTEEVENTS to that of DIAGNOSES_ICD. This proved futile and really reinforced the need for a medical professional to be able to make the correct diagnoses based on their clinical expertise.

Finally, we believe KeyClass may offer the framework to generalize different sets of clinical notes, though more work will need to be done in order to get a version of KeyClass that is streamlined for such a task.


## 4.2 Ablation Studies
Ablation studies are critical for understanding the contribution of individual components of a machine learning model to its overall performance. In the context of KeyClass, we will examine the impact of various features and techniques by altering them one at a time and observing the resultant changes in performance. This approach will help identify which components are essential for the model's effectiveness and efficiency. Below, we will explore the impact of different components of KeyClass on its performance:

### 4.2.1 Activation Functions
Activation functions influence the learning dynamics and convergence behavior of the machine learning models. As a part of our reproduction efforts, we compared performance metrics of KeyClass models with the LeakyReLU activation function against Randomized Leaky ReLU, and Parameteric ReLU functions. Table 2 below shows that Randomized Leaky ReLU edges out both Leaky ReLU (the original model's activation function) and Parametric ReLU, albeit by a narrow margin.
![](https://raw.githubusercontent.com/MinjiaZhu/Tagging-Clinical-Notes-Using-KeyClass/main/Accuracy%20Tables%20-%20Table%202%20Ablation.jpg)

### 4.2.2 Early Stopping
Analysis of model performance with early stopping was NOT included in the original KeyClass model. We explored training two additinal models, one with an early stopping value of two, and another with a value of eight. We compare it to the original model's value of three. The results were somewhat unexpected as the larger early stopping value of eight had slightly more precise results than the orignal model, likely due to overfitting the training model. However, the increase in accuracy was nothing noteworthy. The model did have a rather perceptible increase in accuracy was made when early stopping at a value of two when compared to the other early stopping values. These results felt a bit inconsistent and ultiamtely led us to believe the authors of the original model had landed on a prudent value that did not adversely affect the results of the classification model.

![](https://raw.githubusercontent.com/MinjiaZhu/Tagging-Clinical-Notes-Using-KeyClass/main/Accuracy%20Tables%20-%20Table%203%20Ablation.jpg)


# 5. Discussion

## 5.1 Reproducibility of KeyClass
Reproduction of the KeyClass model proved to be challenging for a variety of reasons. To begin, the original paper didn't clearly explain how to take the raw MIMIC-III data in its original CSV form, to convert it into a dataset that was fully compatible with the KeyClass training model. A model whose configuration files revealed a much more basic structure, with keyword targets and phrases that were less sophisticated than those that would ultimately be used for our model. This proved to be one of the largest hurdles in the model's reproducibility.

The amount of data found in the MIMIC-III dataset wasn't conducive to running sample datasets within a reasonable timeframe, particularly given computational constraints. Limitations within Google Colab Pro often left us without the necessary virtual resources to train large and meaningful sets of data before being disconnected. The scope of the original project was so large that it didn't seem conducive to the purpose of training smaller sample models. We accredit our poor model performance to the simple fact that we needed to parse the data into smaller, more palatable sizes which ultimately may have led to the discrepancy between our efforts, and the efforts of the original authors.

## 5.2 Assessment on Reproducing KeyClass
As expected, there were many challenges encountered when attempting to reproduce this project. The extent of the MIMIC-III data created limitations in terms of understanding what keywords were targeted during the original paper. As previously mentioned, the complexity of the clinical notes seemed to cause issues when attempting to recreate the results of the original paper. Assigning ICD-9 codes to clinical notes was inherently difficult as it is a multiclass and multilabel problem. Without being able to access the original paper's 19-dimensional on-hot vectors, results were likely never going to be similar.

This however should not discredit the potential outlined in the original paper regarding this novel, weakly supervised classification model. With the appropriate dataset, KeyClass does seem to be a powerful tool for simple classification at this point in time. As the results from the original paper indicated, KeyClass outperformed other weakly supervised models like LOTClass (Meng et al., 2020), Dataless (Chang et al., 2008). The potential for this model is apparent.

## 5.3 Recommendations for Reproducibility
The information provided in the KeyClass GitHub repository was a great starting point for this model, however seemingly large and key portions used to train the model were lacking in transparency. The IMDb tutorial provided us with a nice starting point in understanding how KeyClass works, though the authors should have been more thorough in their explanation so future users could follow along a bit more closely without having to make assumptions at key junctures of the modeling.

It would be interesting to see if certain subsets within the healthcare industry would be willing to share their de-identified notes. For example, taking a dataset of clinical notes from within cardiology and training KeyClass for within an ad-hoc sector of healthcare may produce more meaningful results for future iterations of this model.

# References

Gao, C., Goswami, M., Chen, J., Dubrawski, A. "Classifying Unstructured Clinical Notes via Automatic Weak Supervision." Proceedings of Machine Learning Research, 2022, 182:679-690

Bohr, Adam, and Kaveh Memarzadeh. “The Rise of Artificial Intelligence in Healthcare Applications”, Artificial Intelligence in Healthcare, 2020, 25-60, doi:10.1016/B978-0-12-818438-7.00002-2